In [2]:
#导入必要依赖库
import torch.optim
import  torchvision
from torch.nn import Linear, Flatten, MaxPool2d, Conv2d
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch import  nn
from torch.utils.tensorboard import SummaryWriter
import os
import pandas as pd
from PIL import Image
import torch.nn.init as init

In [ ]:
# 文件夹准备
folder_path = '../models'
Train_data_path = '../RAF/images/train'
test_data_path = "../RAF/images/test"
writer = SummaryWriter("./logs_train")

if not os.path.exists(folder_path):
    os.makedirs(folder_path)  

In [4]:
# 查看图片模式和向量大小，方便数据初始化和确认通道数
img = Image.open("../RAF/images/all/0/test_0002_aligned.jpg")
tensor = torchvision.transforms.ToTensor()(img)
print("Tensor 形状：", tensor.shape) 
print("图像模式：", img.mode)

Tensor 形状： torch.Size([3, 100, 100])
图像模式： RGB


In [ ]:
###准备数据集和测试集
###Tensor 形状： torch.Size([3, 100, 100]) 图像模式： RGB
transform = torchvision.transforms.Compose([
     torchvision.transforms.Resize((48,48)),          #统一尺寸
    torchvision.transforms.ToTensor(),                   # 转换为 [C, H, W] 的张量，范围 [0, 1]
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])  
])

train_data = torchvision.datasets.ImageFolder(root=Train_data_path, transform=transform)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True,drop_last=True)

test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False,drop_last=True)

print("类别标签顺序：", train_data.classes)

类别标签顺序： ['0', '1', '2', '3', '4', '5', '6']


In [16]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=7):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        #self.conv1 = nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out



In [ ]:
model = ResNet(BasicBlock, [3,4,5,3], num_classes=7)  
model.cuda()

loss_function = nn.CrossEntropyLoss()
loss_function=loss_function.cuda()

optimizer = torch.optim.AdamW(model.parameters(), 
                              lr=1e-3, 
                              weight_decay=1e-4)


epoch = 15
accuracy = 0
best_acc = 0
accuracies=[]

train_data_len = len(train_data)
test_data_len = len(test_data)
print("训练集的长度为{}".format(train_data_len))
print("测试集的长度为{}".format(test_data_len))

训练集的长度为12271
测试集的长度为3068


In [21]:
total_train_step = 0
torch.cuda.empty_cache()## 训练之前清理显存避免显存爆炸

for i in range(epoch):
    print("---------第{}轮训练开始---------".format(i+1))
    model.train()
    for data in train_loader:
        img,target = data
        img= img.cuda()
        target = target.cuda()
        output = model(img)

        loss=loss_function(output,target)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()
        
        total_train_step+=1

        if total_train_step % 100==0:
            print("训练次数{}，Loss：{}".format(total_train_step,loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
    #测试步骤
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        model.eval()
        for data in test_loader :
            img,target = data
            img = img.cuda()
            target = target.cuda()
            output = model(img)
            loss = loss_function(output,target)
            total_test_loss+=loss.item()
            accuracy = (output.argmax(1) == target ).sum().item()
            total_accuracy += accuracy
    acc=total_accuracy/test_data_len
    print("正确率{}".format(acc))
    accuracies.append(total_accuracy/test_data_len)
    writer.add_scalar("test_loss",total_test_loss,i)
    print("整体测试集上的Loss{}".format(total_test_loss))
    torch.save(model, os.path.join(folder_path, "module_{}.pth".format(i)))
    if(acc>best_acc):
        best_acc = acc

---------第1轮训练开始---------
正确率0.39113428943937417
整体测试集上的Loss36.304271936416626
---------第2轮训练开始---------
训练次数100，Loss：1.7977923154830933
正确率0.5211864406779662
整体测试集上的Loss29.407772034406662
---------第3轮训练开始---------
训练次数200，Loss：1.0619930028915405
正确率0.6095176010430248
整体测试集上的Loss23.499655425548553
---------第4轮训练开始---------
训练次数300，Loss：0.9102106690406799
正确率0.6502607561929595
整体测试集上的Loss20.459740921854973
---------第5轮训练开始---------
训练次数400，Loss：0.7901674509048462
正确率0.6665580182529335
整体测试集上的Loss20.022326186299324
---------第6轮训练开始---------
训练次数500，Loss：0.5231325626373291
正确率0.6603650586701434
整体测试集上的Loss20.370527356863022
---------第7轮训练开始---------
训练次数600，Loss：0.5794152617454529
正确率0.7209908735332464
整体测试集上的Loss17.32255221903324
---------第8轮训练开始---------
训练次数700，Loss：0.557415783405304
正确率0.7095827900912647
整体测试集上的Loss18.109431475400925
---------第9轮训练开始---------
训练次数800，Loss：0.48708590865135193
正确率0.6952411994784876
整体测试集上的Loss18.953563928604126
---------第10轮训练开始---------
训练次数900，Loss：0.

In [22]:
writer.close()